# Tensorflow exercise 
1) Use feature-extraction to train a transfer learning model on 10% of the Food Vision data for 10 epochs using tf.keras.applications.EfficientNetB0 as the base model. Use the ModelCheckpoint callback to save the weights to file.

In [1]:
# lets get the helper function 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-10-09 16:02:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-09 16:02:28 (79.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# lets import the libraries  

import tensorflow as tf 
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,walk_through_dir


In [3]:
# Lets get the 10% food vision data 
# get 10 % of the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2021-10-09 16:02:43--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 142.250.103.128, 108.177.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   152MB/s    in 1.1s    

2021-10-09 16:02:44 (152 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
# unzip the data into the corresponding folder 
unzip_data('10_food_classes_10_percent.zip')

In [5]:
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

In [19]:
# create training and testing directory 
train_dir='10_food_classes_10_percent/train'
test_dir='10_food_classes_10_percent/test'

IMG_SIZE=(224,224)
BATCH_SIZE=32

train_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                              label_mode='categorical',
                                                              image_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE)


test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                             label_mode='categorical',
                                                             image_size=IMG_SIZE,
                                                             batch_size=BATCH_SIZE)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [10]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

# Data augumentation 

Include the data augumentation step into the model. 

In [11]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.layers.experimental import preprocessing

# create data augumentation stage with horizontal flipping , rotations and zooms 

data_augumentation=keras.Sequential([
                                     preprocessing.RandomFlip("horizontal"),
                                     preprocessing.RandomRotation(0.2),
                                     preprocessing.RandomZoom(0.2),
                                     preprocessing.RandomHeight(0.2),
                                     preprocessing.RandomWidth(0.2),
                                     # preprocessing.Rescale(1/255.)
],name='data_augumentation')

In [16]:
input_shape=(224,224,3)
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False 

inputs = tf.keras.Input(shape=input_shape)

x=data_augumentation(inputs)

x=base_model(x,training=False)

x=tf.keras.layers.GlobalAveragePooling2D()(x)

outputs=tf.keras.layers.Dense(len(train_data_10_percent.class_names),activation='softmax')(x)

model_0=keras.Model(inputs,outputs)

In [25]:
checkpoint_path='ten_percent_model_checkpoints_weights/checkpoint.ckpt'
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       save_weights_only=True,
                                                       save_best_only=True,
                                                       monitor='val_loss',
                                                       verbose=1)

In [54]:
intial_epoch=5
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=['accuracy'])

history_0=model_0.fit(train_data_10_percent,
            steps_per_epoch=len(train_data_10_percent),
            validation_steps=(int(len(test_data)*0.25)),
            validation_data=test_data,
            batch_size=BATCH_SIZE,
            epochs=initial_epoch,
            callbacks=[checkpoint_callback])


In [34]:
model_0.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augumentation (Sequenti (None, None, 224)         0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________


In [27]:
model_0.evaluate(test_data)

79/79 [==============================] - 11s 135ms/step - loss: 0.4979 - accuracy: 0.8404


[0.49787065386772156, 0.840399980545044]

Fine-tune the last 20 layers of the base model you trained in 2 for another 10 epochs. How did it go?

In [41]:
# list all the layers and show their status if they are trainable
for layer_number, layers in enumerate(base_model.layers):
  print(layer_number,layers.name,layers.trainable)

0 input_8 False
1 rescaling_4 False
2 normalization_4 False
3 stem_conv_pad False
4 stem_conv False
5 stem_bn False
6 stem_activation False
7 block1a_dwconv False
8 block1a_bn False
9 block1a_activation False
10 block1a_se_squeeze False
11 block1a_se_reshape False
12 block1a_se_reduce False
13 block1a_se_expand False
14 block1a_se_excite False
15 block1a_project_conv False
16 block1a_project_bn False
17 block2a_expand_conv False
18 block2a_expand_bn False
19 block2a_expand_activation False
20 block2a_dwconv_pad False
21 block2a_dwconv False
22 block2a_bn False
23 block2a_activation False
24 block2a_se_squeeze False
25 block2a_se_reshape False
26 block2a_se_reduce False
27 block2a_se_expand False
28 block2a_se_excite False
29 block2a_project_conv False
30 block2a_project_bn False
31 block2b_expand_conv False
32 block2b_expand_bn False
33 block2b_expand_activation False
34 block2b_dwconv False
35 block2b_bn False
36 block2b_activation False
37 block2b_se_squeeze False
38 block2b_se_resha

In [46]:
# set the layer to be trainable 
base_model.trainable=True 
for layer in base_model.layers[:-20]:
  layer.trainable=False



In [47]:


model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_0.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augumentation (Sequenti (None, None, 224)         0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 4,062,381
Trainable params: 1,363,770
Non-trainable params: 2,698,611
_________________________________________________________________


In [49]:
for layers in model_0.layers:
  print(layers,layers.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f5608bae590> True
<keras.engine.sequential.Sequential object at 0x7f5683ba1f90> True
<keras.engine.functional.Functional object at 0x7f5680130850> True
<keras.layers.pooling.GlobalAveragePooling2D object at 0x7f5608bae5d0> True
<keras.layers.core.Dense object at 0x7f568039e3d0> True


In [53]:
for layers in model_0.layers[2].layers:
  print(layers.name,layers.trainable)

input_8 False
rescaling_4 False
normalization_4 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_dwconv False
block2b_bn False
block2b_activation False
block2b_se_squeeze False
block2b_se_reshape False
block2b_se_reduce False
block2b_se_expand False
block2b_se_excite False
block2b_project_conv False

In [52]:
model_0.trainable_variables

[<tf.Variable 'block6d_project_conv/kernel:0' shape=(1, 1, 1152, 192) dtype=float32, numpy=
 array([[[[-0.02386375, -0.0306276 ,  0.08909959, ...,  0.01706599,
            0.03644949, -0.06520524],
          [ 0.02019947,  0.09646644, -0.08918087, ...,  0.00838837,
           -0.00503357, -0.04213767],
          [ 0.05015175, -0.00599662, -0.01823547, ...,  0.02438531,
           -0.14050964,  0.04840524],
          ...,
          [ 0.00757436,  0.00585582,  0.00865441, ...,  0.03820141,
            0.03841049, -0.02001089],
          [ 0.04257223,  0.023178  , -0.00877284, ..., -0.03717634,
            0.06671734,  0.00918923],
          [-0.03928197,  0.06450572, -0.04263911, ..., -0.06215683,
           -0.0602688 ,  0.04657936]]]], dtype=float32)>,
 <tf.Variable 'block6d_project_bn/gamma:0' shape=(192,) dtype=float32, numpy=
 array([1.4481882 , 1.6086447 , 1.6297114 , 1.4362782 , 1.9205517 ,
        1.6919785 , 1.4104973 , 1.9338481 , 1.4338647 , 1.380879  ,
        2.4113564 , 1.5

In [57]:
# fit the model zero 
epochs=intial_epoch+10
history_0_last_10=model_0.fit(train_data_10_percent,
                              steps_per_epoch=len(train_data_10_percent),
                              validation_data=test_data,
                              validation_steps=int(len(test_data)*0.25),
                              batch_size=32,
                              initial_epoch=history_0.epoch[-1],
                              callbacks=[checkpoint_callback],
                              epochs=epochs)

Epoch 5/15
24/24 [==============================] - 24s 620ms/step - loss: 0.5802 - accuracy: 0.7987 - val_loss: 0.5614 - val_accuracy: 0.8355

Epoch 00005: val_loss did not improve from 0.44336
Epoch 6/15
24/24 [==============================] - 12s 486ms/step - loss: 0.2517 - accuracy: 0.9280 - val_loss: 0.4565 - val_accuracy: 0.8684

Epoch 00006: val_loss did not improve from 0.44336
Epoch 7/15
24/24 [==============================] - 11s 465ms/step - loss: 0.1838 - accuracy: 0.9453 - val_loss: 0.4631 - val_accuracy: 0.8602

Epoch 00007: val_loss did not improve from 0.44336
Epoch 8/15
24/24 [==============================] - 11s 457ms/step - loss: 0.1493 - accuracy: 0.9533 - val_loss: 0.4573 - val_accuracy: 0.8602

Epoch 00008: val_loss did not improve from 0.44336
Epoch 9/15
24/24 [==============================] - 12s 501ms/step - loss: 0.1039 - accuracy: 0.9707 - val_loss: 0.4981 - val_accuracy: 0.8668

Epoch 00009: val_loss did not improve from 0.44336
Epoch 10/15
24/24 [======